In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors

In [ ]:
# z ~ N(0, I)
# x[i] = z[i] * scale(x[0: i-1]) + loc(x[0: i-1]), i=0,...,D-1

Let's say that our data is D dimensional and we are representing it as the vector x. Also let's say that we've defined an ordering of the data features, which are indexed by i. In an autoregressive model we express the joint distribution of all data features as a product of conditional distributions, where each conditional probability of the feature x_i depends only on the features x_0 to x_i-1. In the example shown here, we are modeling the conditional distribution of each feature x_i as a Gaussian distribution, where the mean and standard deviation are function of the features x_0 to x_i-1. This autoregressive model is called a masked autoregressive flow. One important property of this flow is that the jacobian determinant is easy to compute because the jacobian matrix is lower triangular thanks to the autoregressive property. The log determinant of the jacobian of the forward transformation is just the negative sum of the log standard deviations for each feature.

In [3]:
made = tfb.AutoregressiveNetwork(params=2,
                                event_shape=[3],
                                hidden_units=[16, 16],
                                activation='sigmoid')

The autoregressive network is a feed forward neural network which in our case has 2 hidden layers of size 16 and a sigmoid activation function. The input size of the network can be defined by the using the event_shape. It returns an output that has the same shape as the input plus an extra dimension. This extra dimension has size given by the params argument.

In [4]:
made(tf.random.normal([2, 3]))

<tf.Tensor: shape=(2, 3, 2), dtype=float32, numpy=
array([[[ 0.        ,  0.        ],
        [-0.960885  , -0.35263348],
        [ 0.6401592 , -0.67108166]],

       [[ 0.        ,  0.        ],
        [-0.96432364, -0.35303643],
        [ 0.6791935 , -0.68823755]]], dtype=float32)>

So if the pass an input of shape (2, 3), which means a batch of two input data points each of length 3, the returned output will have shape of (2,3,2). The last 2 is set by the params argument.

You can look at this network as implementing the loc and scale functions. For each element in a batch the network is outputing 2 parameters each of length 3. We can use the first of these to the mean and the second for the log of the standard deviation. So we have a mean parameter for each data feature and a standard deviation parameter for each data feature. This network is designed to respect the autoregressive property. This means that each parameter for each feature i is computed oly using the features up to i-1. It does this by zeroing out a number of weights in the network.

In [5]:
maf_bijector = tfb.MaskedAutoregressiveFlow(shift_and_log_scale_fn=made)

Now that we have our autoregressive network, we can use it to define the masked autoregressive flow that was described at the top. We want to use the parameters outputed by our made network to shift and scale the random variables z_i. The bijector that does this for us is the masked autoregressive flow bijector and this bijector will compute precisely the transformation above by passing the made autoregressive network into the shift_and_log_scale_fn keywork argument.

In [6]:
def forward(z):
    x = tf.zeros_like(z)
    for _ in range(D):
        shift, log_scale = shift_and_log_scale_fn(x)
        x = z * tf.math.exp(log_scale) + shift
    return z

The pseudocode above gives an idea of what is happening in the forward transformation. Given a realization of the random variable z we first initialize a vector x. Then we need to iterate over each feature. This loop implements the equation written at the top. The made network, which is the shift_and_log_scale_function used inside the loop, outputs the mean and log standard deviation for each feature and that the parameters for each feature depend only on the features that come before it. That means that in the first pass through this loop, the mean and scale paramters for the first feature will be correct as they don't depend on any other feature. That means that the first entry in the vector x will be correct after the first pass. Then, on the second pass, the mean and scale paramters will be correct for the second feature. Now, the first two features in vector x will be correct. After D passes through the loop, all features in the vector x will be updated correctly.

In [7]:
def inverse(x):
    shift, log_scale = shift_and_log_scale_fn(x)
    return (x - shift) / tf.math.exp(log_scale)

The inverse transformation is simpler and that is because no loops are required. If we have all the vector x available, we can solve the equation at the top for all values of z in parallel. That is what is written on the pseudocode above. We can compute all mean and scale parameters in one go and use them to compute the z variable in parallel.

In [ ]:
normal = tfd.Normal(loc=0, scale=1)
maf = tfd.TransformedDistribution(normal, maf_bijector, event_shape=[3])

Now that we have our autoregressive flow bijector setup, we can now define our transformed distribution as before using the TransformedDistribution object. We pass in the base distribution, which is just a standard normal and the masked autoregressive flow bijector. We pass in an event_shape of 3 on the transformed distribution as the bijector is operating on a vector event of size 3. We can now sample from the transformed distribution or compute the log probability. Remember that sampling from this distribution will evoke the forward method, which involves the loop over all the features. For data with a large number of features and for a model with a large autoregressive network, this can be very slow. But in contrast, computing log probabilities is fast because it uses the inverse transformation, which, as we've seen, can be done in parallel.

In [9]:
# z ~ N(0, I)
# x[i] = z[i] + loc(x[0: i-1]), i=0,...,D-1

maf_bijector = tfb.MaskedAutoregressiveFlow(
    lambda y: (made(y)[..., 0], None),
    is_constant_jacobian=True
)

We can modify various aspects of this autoregressive flow. For example we can define a autoregressive network that does not scale the random variable z, only shifts it. Above we are using the same autoregressive network for the shift and log scale function but in the argument to the maked autoregressive flow constructor we are passing in a lambda function that only uses the first of these parameters and passing None for the log scale parameter. This also means that the jacobian for this transformation will be constant. In fact, it will be the indentity and so we are passing is_constant_jacobian=True into the constructor as well.

In [ ]:
iaf_bijector = tfb.Invert(tfb.MaskedAutoregressiveFlow(shift_and_log_scale_fn=made))
normal = tfd.Normal(loc=0, scale=1)
maf = tfd.TransformedDistribution(normal, iaf_bijector, event_shape=[3])

Finally, we can use the masked autoregressive flow bijector to define an inverse autoregressive flow. We only need to apply the invert bijector to a masked autoregressive flow object. Notice that the inverse autoregressive flow is slow when calculating the log probability and fast at sampling. 